In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ops.ops import load_json, save_json
import ee
#import geemap
import fiona
import time
#import json
import os
import datetime
from tqdm.notebook import tqdm_notebook

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWgY2nXFcJAq4lw59O94Q2-WP0N5XHNwf2il1cTSnOqMAYJSJqm0_bs

Successfully saved authorization token.


In [7]:
Map = geemap.Map(center=(-23, -45), zoom=8)
Map

Map(center=[-23, -45], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [9]:
paths = load_json(os.path.join('conf', 'paths.json'))
shp = load_json(os.path.join('conf', 'shp.json'))
shp_path = paths['shp']
max_cloud_cover = 95
max_thin_cirrus = 0.1
delta_days = 10
meta_path = os.path.join('img', 'metadata')

In [10]:
source_file = shp['shp_base']
grid_source = os.path.join(shp_path, f'{source_file}.shp')

In [25]:
def prep_fn(date):
    def diff_date(image):
        return image.set(
        'dateDist',
        ee.Number(image.get('system:time_start')).subtract(datetime.datetime.strptime(date, '%Y-%m-%d').timestamp()*1000).abs()
      )
    return diff_date

In [20]:
def test_date(day, year, img_coll):
    date_i = datetime.datetime.strptime(f'{year}{day}', '%y%j').strftime('%Y-%m-%d')
    date_f = datetime.datetime.strptime(f'{year}{day+1}', '%y%j').strftime('%Y-%m-%d')

    img_coll_t = img_coll.filterBounds(roi).filterDate(date_i, date_f)
    #print(d19_t)
    #print(opt_col_2019_t.size().getInfo())
    if img_coll_t.size().getInfo() > 0:
        min_mask = img_coll_t.mean().select('B2').mask().reduceRegion(ee.Reducer.min(), roi, 10).getInfo()['B2']
        if min_mask == 1:
            return False
        if min_mask == 0:
            return True
    return False

## Evaluate L-2A images

In [34]:
dest_file = shp['shp_imgs_l2']
grid_dest = os.path.join(shp_path, f'{dest_file}.shp')

with fiona.open(grid_source) as grid:
    source_driver = grid.driver
    source_crs = grid.crs
    source_schema = grid.schema
    
    #adding new shapefile columns
    source_schema['properties'].update({'cloud_2019':'float:24.15'})
    source_schema['properties'].update({'cloud_2020':'float:24.15'})
    source_schema['properties'].update({'cloud_max':'float:24.15'})
    source_schema['properties'].update({'opt_2019':'str:254'})
    source_schema['properties'].update({'opt_2020':'str:254'})
    source_schema['properties'].update({'sar_2019':'str:254'})
    source_schema['properties'].update({'sar_2020':'str:254'})
    source_schema['properties'].update({'crs':'str:254'})
    
    with fiona.open(grid_dest, 'w', driver=source_driver, crs=source_crs, schema=source_schema) as dest:
        
        for feat in tqdm_notebook(grid, desc = 'Tiles evaluated'):
            roi = ee.Geometry(feat['geometry'])
            feat_id = int(feat['properties']['id'])

            d19 = int(feat['properties']['jday_2019'])
            d20 = int(feat['properties']['jday_2020'])
            d19_1 = d19-delta_days
            d19_2 = d19+delta_days
            d20_1 = d20-delta_days
            d20_2 = d20+delta_days

            d19 = datetime.datetime.strptime(f'19{d19}', '%y%j').strftime('%Y-%m-%d')
            d20 = datetime.datetime.strptime(f'20{d20}', '%y%j').strftime('%Y-%m-%d')
            d19_1 = datetime.datetime.strptime(f'19{d19_1}', '%y%j').strftime('%Y-%m-%d')
            d19_2 = datetime.datetime.strptime(f'19{d19_2}', '%y%j').strftime('%Y-%m-%d')
            d20_1 = datetime.datetime.strptime(f'20{d20_1}', '%y%j').strftime('%Y-%m-%d')
            d20_2 = datetime.datetime.strptime(f'20{d20_2}', '%y%j').strftime('%Y-%m-%d')

            opt_col_2019 = ee.ImageCollection('COPERNICUS/S2_SR').filterDate(d19_1, d19_2)
            opt_col_2020 = ee.ImageCollection('COPERNICUS/S2_SR').filterDate(d20_1, d20_2)
            sar_col_2019 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(d19_1, d19_2)
            sar_col_2020 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(d20_1, d20_2)
            
            opt_col_2019 = opt_col_2019.map(prep_fn(d19))
            opt_col_2020 = opt_col_2020.map(prep_fn(d20))
            sar_col_2019 = sar_col_2019.map(prep_fn(d19))
            sar_col_2020 = sar_col_2020.map(prep_fn(d20))

            #print(f'\n\nlocating images from grid {feat_id}...')

            opt_2019 = opt_col_2019.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',max_cloud_cover))
            #opt_2019 = opt_col_2019.filter(ee.Filter.lt('THIN_CIRRUS_PERCENTAGE',max_thin_cirrus))
            opt_2019 = opt_2019.filterBounds(roi)
            opt_2019 = opt_2019.filter(ee.Filter.contains('.geo', roi))

            opt_2020 = opt_col_2020.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',max_cloud_cover))
            #opt_2020 = opt_col_2020.filter(ee.Filter.lt('THIN_CIRRUS_PERCENTAGE',max_thin_cirrus))
            opt_2020 = opt_2020.filterBounds(roi)
            opt_2020 = opt_2020.filter(ee.Filter.contains('.geo', roi))

            sar_2019 = sar_col_2019.filterBounds(roi)
            sar_2019 = sar_2019.filter(ee.Filter.contains('.geo', roi))
            sar_2020 = sar_col_2020.filterBounds(roi)
            sar_2020 = sar_2020.filter(ee.Filter.contains('.geo', roi))
            #print(f'counting images from grid {feat_id}...')

            n_opt_2019 = opt_2019.size().getInfo()
            n_opt_2020 = opt_2020.size().getInfo()
            n_sar_2019 = sar_2019.size().getInfo()
            n_sar_2020 = sar_2020.size().getInfo()

            #print(f'Number of images from grid {feat_id}: OPT_2019:{n_opt_2019} | OPT_2020:{n_opt_2020} | SAR_2019:{n_sar_2019} | SAR_2020:{n_sar_2020} ')
            
            if n_opt_2019 == 0 or n_opt_2020 == 0 or n_sar_2019 == 0 or n_sar_2020 == 0:
                continue
            
            opt_2019 = opt_2019.sort('dateDist').first()
            opt_2019_pm = opt_2019.select(['MSK_CLDPRB']).clip(roi)
            
            opt_2020 = opt_2020.sort('dateDist').first()
            opt_2020_pm = opt_2020.select(['MSK_CLDPRB']).clip(roi)
            max_pm = opt_2019_pm.max(opt_2020_pm)
            
            sar_2019 = sar_2019.sort('dateDist').first()
            sar_2020 = sar_2020.sort('dateDist').first()

            mean_opt_2019 =  opt_2019_pm.reduceRegion(ee.Reducer.mean()).getInfo()['MSK_CLDPRB']
            mean_opt_2020 =  opt_2020_pm.reduceRegion(ee.Reducer.mean()).getInfo()['MSK_CLDPRB']
            max_opt =  max_pm.reduceRegion(ee.Reducer.mean()).getInfo()['MSK_CLDPRB']

            feat['properties']['cloud_2019'] = mean_opt_2019
            feat['properties']['cloud_2020'] = mean_opt_2020
            feat['properties']['cloud_max'] = max_opt
            feat['properties']['opt_2019'] = opt_2019.getInfo()['id']
            feat['properties']['opt_2020'] = opt_2020.getInfo()['id']
            feat['properties']['sar_2019'] = sar_2019.getInfo()['id']
            feat['properties']['sar_2020'] = sar_2020.getInfo()['id']
            feat['properties']['crs'] = opt_2019.getInfo()['bands'][1]['crs']
            
            dest.write(feat)
            
            meta_opt_2019 = os.path.join(meta_path, f'{feat_id}_opt_2019.txt')
            meta_opt_2020 = os.path.join(meta_path, f'{feat_id}_opt_2020.txt')
            meta_sar_2019 = os.path.join(meta_path, f'{feat_id}_sar_2019.txt')
            meta_sar_2020 = os.path.join(meta_path, f'{feat_id}_sar_2020.txt')
            
            save_json(opt_2019.getInfo(), meta_opt_2019)
            save_json(opt_2020.getInfo(), meta_opt_2020)
            save_json(sar_2019.getInfo(), meta_sar_2019)
            save_json(sar_2020.getInfo(), meta_sar_2020)
        

Tiles evaluated:   0%|          | 0/255 [00:00<?, ?it/s]

## Evaluate L-1C images

In [35]:
dest_file = shp['shp_imgs_l1']
grid_dest = os.path.join(shp_path, f'{dest_file}.shp')

with fiona.open(grid_source) as grid:
    source_driver = grid.driver
    source_crs = grid.crs
    source_schema = grid.schema
    
    #adding new shapefile columns
    source_schema['properties'].update({'opt_2019':'str:254'})
    source_schema['properties'].update({'opt_2020':'str:254'})
    source_schema['properties'].update({'sar_2019':'str:254'})
    source_schema['properties'].update({'sar_2020':'str:254'})
    source_schema['properties'].update({'crs':'str:254'})
    
    with fiona.open(grid_dest, 'w', driver=source_driver, crs=source_crs, schema=source_schema) as dest:
        
        for feat in tqdm_notebook(grid, desc = 'Tiles evaluated'):
            roi = ee.Geometry(feat['geometry'])
            feat_id = int(feat['properties']['id'])

            d19 = int(feat['properties']['jday_2019'])
            d20 = int(feat['properties']['jday_2020'])
            d19_1 = d19-delta_days
            d19_2 = d19+delta_days
            d20_1 = d20-delta_days
            d20_2 = d20+delta_days

            d19 = datetime.datetime.strptime(f'19{d19}', '%y%j').strftime('%Y-%m-%d')
            d20 = datetime.datetime.strptime(f'20{d20}', '%y%j').strftime('%Y-%m-%d')
            d19_1 = datetime.datetime.strptime(f'19{d19_1}', '%y%j').strftime('%Y-%m-%d')
            d19_2 = datetime.datetime.strptime(f'19{d19_2}', '%y%j').strftime('%Y-%m-%d')
            d20_1 = datetime.datetime.strptime(f'20{d20_1}', '%y%j').strftime('%Y-%m-%d')
            d20_2 = datetime.datetime.strptime(f'20{d20_2}', '%y%j').strftime('%Y-%m-%d')

            opt_col_2019 = ee.ImageCollection('COPERNICUS/S2').filterDate(d19_1, d19_2)
            opt_col_2020 = ee.ImageCollection('COPERNICUS/S2').filterDate(d20_1, d20_2)
            sar_col_2019 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(d19_1, d19_2)
            sar_col_2020 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(d20_1, d20_2)
            
            opt_col_2019 = opt_col_2019.map(prep_fn(d19))
            opt_col_2020 = opt_col_2020.map(prep_fn(d20))
            sar_col_2019 = sar_col_2019.map(prep_fn(d19))
            sar_col_2020 = sar_col_2020.map(prep_fn(d20))

            #print(f'\n\nlocating images from grid {feat_id}...')

            opt_2019 = opt_col_2019.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',max_cloud_cover))
            #opt_2019 = opt_col_2019.filter(ee.Filter.lt('THIN_CIRRUS_PERCENTAGE',max_thin_cirrus))
            opt_2019 = opt_2019.filterBounds(roi)
            opt_2019 = opt_2019.filter(ee.Filter.contains('.geo', roi))

            opt_2020 = opt_col_2020.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',max_cloud_cover))
            #opt_2020 = opt_col_2020.filter(ee.Filter.lt('THIN_CIRRUS_PERCENTAGE',max_thin_cirrus))
            opt_2020 = opt_2020.filterBounds(roi)
            opt_2020 = opt_2020.filter(ee.Filter.contains('.geo', roi))

            sar_2019 = sar_col_2019.filterBounds(roi)
            sar_2019 = sar_2019.filter(ee.Filter.contains('.geo', roi))
            sar_2020 = sar_col_2020.filterBounds(roi)
            sar_2020 = sar_2020.filter(ee.Filter.contains('.geo', roi))
            #print(f'counting images from grid {feat_id}...')

            n_opt_2019 = opt_2019.size().getInfo()
            n_opt_2020 = opt_2020.size().getInfo()
            n_sar_2019 = sar_2019.size().getInfo()
            n_sar_2020 = sar_2020.size().getInfo()

            #print(f'Number of images from grid {feat_id}: OPT_2019:{n_opt_2019} | OPT_2020:{n_opt_2020} | SAR_2019:{n_sar_2019} | SAR_2020:{n_sar_2020} ')
            
            if n_opt_2019 == 0 or n_opt_2020 == 0 or n_sar_2019 == 0 or n_sar_2020 == 0:
                continue

            opt_2019 = opt_2019.sort('dateDist').first()
            
            opt_2020 = opt_2020.sort('dateDist').first()
            
            sar_2019 = sar_2019.sort('dateDist').first()
            sar_2020 = sar_2020.sort('dateDist').first()


            feat['properties']['opt_2019'] = opt_2019.getInfo()['id']
            feat['properties']['opt_2020'] = opt_2020.getInfo()['id']
            feat['properties']['sar_2019'] = sar_2019.getInfo()['id']
            feat['properties']['sar_2020'] = sar_2020.getInfo()['id']
            feat['properties']['crs'] = opt_2019.getInfo()['bands'][1]['crs']
            
            dest.write(feat)
            
            meta_opt_2019 = os.path.join(meta_path, f'{feat_id}_opt_2019.txt')
            meta_opt_2020 = os.path.join(meta_path, f'{feat_id}_opt_2020.txt')
            meta_sar_2019 = os.path.join(meta_path, f'{feat_id}_sar_2019.txt')
            meta_sar_2020 = os.path.join(meta_path, f'{feat_id}_sar_2020.txt')
            
            save_json(opt_2019.getInfo(), meta_opt_2019)
            save_json(opt_2020.getInfo(), meta_opt_2020)
            save_json(sar_2019.getInfo(), meta_sar_2019)
            save_json(sar_2020.getInfo(), meta_sar_2020)
        

Tiles evaluated:   0%|          | 0/255 [00:00<?, ?it/s]